In [ ]:
#****************************************ETIHAD CARGO MESSAGE PARSER CHALLENGE*************************************************#

In [ ]:
##############################################TEAM MEMBERS###########################################################################


# 1.SHOYEB KHAN                -2021H1540806P
# 2.SALMAN AHMED KHAN          -2021H1540830P
# 3.MIRA PRIYADARSHINI K       -2021H1540817P

In [ ]:
#######################################OBJECTIVE###############################################################################

#To develop a Message Parser with the following features:
#   a.To process Cargo EDI Messages in a specified format.
#   b.To identify and log any errors encountered during the processing of the message.
#The data is sourced from Etihad Cargo company

#In simple terms,message parsing is basically splitting the input message into smaller components so that the meaning of the message can be understood.

In [ ]:
## Saving all the files in one location.
import os, sys
from google.colab import drive
drive.mount('/content/mnt')
nb_path = '/content/notebooks'
os.symlink('/content/mnt/My Drive/Colab Notebooks', nb_path)
sys.path.insert(0, nb_path)

Mounted at /content/mnt


In [ ]:
#!pip install --target=$nb_path fasttext
#!pip install --target=$nb_path pattern
#!pip install --target=$nb_path scikit-learn
#!pip install --target=$nb_path -U scikit-learn
#!pip install --target=$nb_path -U regex
#!pip install --target=$nb_path nltk
#!pip install black --no-binary=regex
#!pip install --user -U nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 772 kB 4.7 MB/s 


In [ ]:
import regex

In [ ]:
## We have to build a classification model based on the facebook's FastText library.

In [ ]:
###What is fastText?
###FastText is an open-source, free, lightweight library that allows users to learn text representations and
#text classifiers. It works on standard, generic hardware. Models can later be reduced in size to even fit on mobile devices.

In [ ]:
import fasttext
import pandas as pd
import re

from sklearn import feature_extraction, linear_model, model_selection, preprocessing

In [ ]:
## Bring the text data into the colab space.
! gdown --id 1bU_LWVC1wRm_eMUXhQpJfQGIFbjX8UuA
! gdown --id 1nu9VHN1hc6hPgsFVsWwgTJ44vWu2hNKr
! gdown --id 134g1coD06ILJaBsP7iHZm27hlWlkWs-Z

/usr/local/lib/python3.8/dist-packages/gdown/cli.py:127: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1bU_LWVC1wRm_eMUXhQpJfQGIFbjX8UuA
To: /content/CPM1.txt
100% 16.8k/16.8k [00:00<00:00, 19.3MB/s]
/usr/local/lib/python3.8/dist-packages/gdown/cli.py:127: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1nu9VHN1hc6hPgsFVsWwgTJ44vWu2hNKr
To: /content/CPM1_f.txt
100% 1.43k/1.43k [00:00<00:00, 976kB/s]
/usr/local/lib/python3.8/dist-packages/gdown/cli.py:127: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=134g

In [ ]:
cpm_correct_format = pd.read_csv('/content/CPM1_f.txt',error_bad_lines=False,header=None)
cpm_incorrect_format = pd.read_csv('/content/CPM1_incorrect.txt',error_bad_lines=False,header=None)

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
cpm_correct_format.to_csv('cpm_correct_format.txt',index = False,sep =' ',header = None, quoting =None,quotechar = " ",escapechar = " ")
cpm_incorrect_format.to_csv('cpm_incorrect_format.txt',index = False,sep =' ',header = None, quoting =None,quotechar = " ",escapechar = " ")

In [ ]:
cpm_correct_format_txt = pd.read_csv('cpm_correct_format.txt',header = None)
cpm_incorrect_format_txt = pd.read_csv('cpm_incorrect_format.txt',header = None)

In [ ]:
cpm_correct_format_txt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45 entries, 0 to 44
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       45 non-null     object
dtypes: object(1)
memory usage: 488.0+ bytes


In [ ]:
cpm_incorrect_format_txt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 0 to 51
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       52 non-null     object
dtypes: object(1)
memory usage: 544.0+ bytes


In [ ]:
#Split into train and test data set.60 percent of data is training data and 40 percent is testing data.
txt_file = pd.read_csv('/content/CPM1.txt',error_bad_lines=False,header=None)
train_file = txt_file[txt_file.index <= 0.6*max(txt_file.index)]
test_file = txt_file[txt_file.index > 0.6*max(txt_file.index)]

b'Skipping line 94: expected 1 fields, saw 2\nSkipping line 223: expected 1 fields, saw 8\n'


In [ ]:
train_file.to_csv('train.txt',index = False,sep =' ',header = None, quoting =None,quotechar = " ",escapechar = " ")
test_file.to_csv('test.txt',index = False,sep =' ',header = None, quoting =None,quotechar = " ",escapechar = " ")

In [ ]:
train_txt = pd.read_csv('train.txt',header = None)
test_txt = pd.read_csv('test.txt',header = None)

In [ ]:
train_txt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 216 entries, 0 to 215
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       216 non-null    object
dtypes: object(1)
memory usage: 1.8+ KB


In [ ]:
##The training data has 216 entries of messages
##The training data is grouped into message and label

In [ ]:
label_ = []
message = []

for i in range(0,len(train_txt)):
  mssg = str(train_txt[train_txt.index==i])
  mssg_ = re.split(r"\s+", mssg,maxsplit=4)
  label_.append(mssg_[3])
  message.append(mssg_[4])

In [ ]:
##A dataframe for training data is created

In [ ]:
train_df = pd.DataFrame({'Message':message,'Label':label_})

In [ ]:
###The value counts of the label is measured.
###It is identified that 177 messages are without any errors(syntax/data errors).
###It is identified that 39 messages are having errors(syntax/data errors).


In [ ]:
train_df['Label'].value_counts()

__label__no_error    177
__label__error        39
Name: Label, dtype: int64

In [ ]:
##The testing data has 144 entries of messages
##The testing data is grouped into message and label

In [ ]:
label_ = []
message = []

for i in range(0,len(test_txt)):
  mssg = str(test_txt[test_txt.index==i])
  mssg_ = re.split(r"\s+", mssg,maxsplit=4)
  label_.append(mssg_[3])
  message.append(mssg_[4])

In [ ]:
##A dataframe for testing data is created

In [ ]:
test_df = pd.DataFrame({'Message':message,'Label':label_})

In [ ]:
###The value counts of the label is measured.
###It is identified that 137 messages are without any errors(syntax/data errors).
###It is identified that 7 messages are having errors(syntax/data errors).


In [ ]:
test_df['Label'].value_counts()

__label__no_error    137
__label__error         7
Name: Label, dtype: int64

In [ ]:
############################################################# Model 1 Based on Fast Text##################################################################################

In [ ]:
#The Message Parser is trained using the train_supervised function of fasttext library.The training is done using the training text data.

In [ ]:
model = fasttext.train_supervised(input='train.txt')

In [ ]:
model.test('test.txt')

(144, 0.9513888888888888, 0.9513888888888888)

In [ ]:
#Upon testing the message parser using the test data,it is observed that the accuracy of the model is 95%.

In [ ]:
## Add more epochs and larger learning rate. This is just to see if the accuracy is improved or not.
## Epoch:A hyperparameter that defines the number times that the learning algorithm will work through the entire training dataset. One epoch means that each sample in the training dataset has had an opportunity to update the internal model parameters.
## Learning rate: It is a tuning parameter in an optimization algorithm that determines the step size at each iteration while moving toward a minimum of a loss function.
## wordNgrams : max length of word ngram

In [ ]:
model_1 = fasttext.train_supervised(input="train.txt", epoch=25,lr=0.95,wordNgrams=3)

In [ ]:
model_1.test("test.txt")

(144, 0.9513888888888888, 0.9513888888888888)

In [ ]:
model.get_labels()

In [ ]:
##After adding more epochs and larger learning rate ,the accuracy of the model is still 95%

In [ ]:
######################################################################Model 2 Using ML Algo###############################################################################

In [ ]:
##CountVectorizer is a great tool provided by the scikit-learn library in Python. It is used to transform a given text into a vector on the basis of the frequency (count) of each word that occurs in the entire text.

In [ ]:
count_vectorizer = feature_extraction.text.CountVectorizer()

In [ ]:
##The training and testing messages are transformed into vectors using the count_vectorizer method

In [ ]:
train_vectors = count_vectorizer.fit_transform(train_df["Message"])
test_vectors = count_vectorizer.fit_transform(test_df["Message"])

In [ ]:
##The training and testing data are given label numbers using lambda function.
##If the label is "__label__no_error",then label number will be 0.
##If the label is "__label__error",then label number will be 1.


In [ ]:
train_df['Label_num'] = train_df['Label'].apply(
    lambda x: 1 if x == '__label__error' else (0 if x == '__label__no_error' else None))

In [ ]:
test_df['Label_num'] = test_df['Label'].apply(
    lambda x: 1 if x == '__label__error' else (0 if x == '__label__no_error' else None))

In [ ]:
##Ridge classification is a technique used to analyze linear discriminant models. It is a form of regularization that penalizes model coefficients to prevent overfitting
##Overfitting is a common issue in machine learning that occurs when a model is too complex and captures noise in the data instead of the underlying signal.
##Ridge classification works by adding a penalty term to the cost function that discourages complexity. The penalty term is typically the sum of the squared coefficients of the features in the model. This forces the coefficients to remain small, which prevents overfitting.

In [ ]:
##cross_val_score():It is a method which runs cross validation on a dataset to test whether the model can generalise over the whole dataset. The function returns a list of one score per split, and the average of these scores can be calculated to provide a single metric value for the dataset.
##This is a function and a technique which you should add to your workflow to make sure you are developing highly performant models.
##The parameters passed in this method are estimator:clf,the data to fit the model on,the target of the model,number of splits to use:3,scoring metric:f1 score

In [ ]:
clf = linear_model.RidgeClassifier()
scores = model_selection.cross_val_score(clf, train_vectors, train_df["Label_num"], cv=3, scoring="f1")
scores

array([0.66666667, 0.66666667, 0.14285714])

In [ ]:
##The f1-scores of the 3 splits are 66.7%,66.7% and 14.3% respectively.

In [ ]:
#################################################### Model 3 Using Bigram & Trigram #####################################################################################

In [ ]:
train_df.head()

,Message,Label,Label_num
0,CPM,__label__no_error,0
1,EY972/11.A6DDD.HAN,__label__no_error,0
2,-PL/PMC71439EY/AUH/2223/C....,__label__no_error,0
3,-PR/PMC74892EY/AUH/2370/C....,__label__no_error,0
4,-R/PMC72439EY/AUH/985/C.EL...,__label__no_error,0


In [ ]:
train_no_error = train_df[train_df['Label']=='__label__no_error']

In [ ]:
train_no_error.head()

,Message,Label,Label_num
0,CPM,__label__no_error,0
1,EY972/11.A6DDD.HAN,__label__no_error,0
2,-PL/PMC71439EY/AUH/2223/C....,__label__no_error,0
3,-PR/PMC74892EY/AUH/2370/C....,__label__no_error,0
4,-R/PMC72439EY/AUH/985/C.EL...,__label__no_error,0


In [ ]:
##NLTK is a toolkit build for working with NLP in Python. It provides us various text processing libraries with a lot of test datasets. A variety of tasks can be performed using NLTK such as tokenizing, parse tree visualization, etc.
##Punkt Sentence Tokenizer : This tokenizer divides a text into a list of sentences by using an unsupervised algorithm to build a model for abbreviation words, collocations, and words that start sentences. It must be trained on a large collection of plaintext in the target language before it can be used.

In [ ]:
import regex

In [ ]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
##The training messages are tokenized into sentences.
##The sentences are then tokenized into words.

In [ ]:
sentences = train_no_error['Message'].tolist()

In [ ]:
sentences_correct = cpm_correct_format_txt[0].tolist()

In [ ]:
tokenized_sent = []
for s in sentences:
    tokenized_sent.append(word_tokenize(s.lower()))
print(tokenized_sent)

[['cpm'], ['ey972/11.a6ddd.han'], ['-pl/pmc71439ey/auh/2223/c', '....'], ['-pr/pmc74892ey/auh/2370/c', '....'], ['-r/pmc72439ey/auh/985/c.el', '...'], ['-11p/pmc01336ey/auh/1165/c', '...'], ['-12p/pmc75635ey/auh/1860/c', '...'], ['-13p/pmc70016ey/auh/2050/c', '...'], ['-21p/pmc76368ey/auh/2270/c', '...'], ['-22p/pmc72223ey/auh/2375/c', '...'], ['-23p/pmc74605ey/auh/2390/c', '...'], ['-31p/pmc74776ey/auh/2445/c', '...'], ['-33l/ake29519ey/auh/444/c', '....'], ['-33r/ake29217ey/auh/65/x'], ['-41l/ake26910ey/auh/65/x'], ['-41r/n'], ['-41p/pmc75812ey/auh/2475/c', '...'], ['-42p/pmc74004ey/auh/2405/c', '...'], ['-51/n'], ['-52/n'], ['-53/n'], ['cpm'], ['ey956/28.a6ddd.auh'], ['-ar/pmc77270ey/jnb/1540/c'], ['-bl/pmc77913ey/jnb/1235/c'], ['-br/pmc77717ey/jnb/1250/c'], ['-cl/pmc70333ey/jnb/1420/c'], ['-cr/pmc75544ey/jnb/1440/c', '....'], ['-dl/pmc76410ey/jnb/1475/c'], ['-dr/pmc76297ey/jnb/1605/c'], ['-el/pmc77378ey/jnb/1770/c'], ['-er/pmc76550ey/nbo/1820/c'], ['-fl/pmc72704ey/jnb/1920/c', '...

In [ ]:
tokenized_sent = []
for s in sentences_correct:
    tokenized_sent.append(word_tokenize(s.lower()))
print(tokenized_sent)

[['cpm'], ['ey972/11.a6ddd.han'], ['-al/pmc77612ey/auh/1915/c.eli.vun'], ['-ar/pmc72671ey/auh/1840/c.eli.vun'], ['-bl/pmc76614ey/auh/1705/c.eli.vun'], ['-br/pmc76649ey/auh/1705/c.eli.vun'], ['-cl/pmc75672ey/auh/2275/c.eli.hea.vun'], ['-cr/pmc76847ey/auh/1855/c.eli.vun'], ['-dl/pmc74817ey/auh/2490/c.eli.elm.hea.vun'], ['-dr/pmc01084ey/auh/2460/c.eli.vun'], ['-el/pmc76217ey/auh/2675/c.eli.vun'], ['-er/pmc74925ey/auh/2655/c.eli.elm.hea.vun'], ['-fl/pmc77135ey/auh/2685/c.eli.vun'], ['-fr/pmc77284ey/auh/2660/c.eli.vun'], ['-gl/pmc71204ey/auh/2730/c.eli.vun'], ['-gr/pmc76143ey/auh/2860/c.eli.vun'], ['-hl/pmc77707ey/auh/3035/c.eli.vun'], ['-hr/pmc74214ey/auh/3115/c.eli.vun'], ['-jl/pmc70639ey/auh/3475/c.eli.vun'], ['-jr/pmc76874ey/auh/3820/c.eli.vun'], ['-kl/pmc70953ey/auh/4060/c.eli.vun'], ['-kr/pmc79889ey/auh/4090/c.eli.vun'], ['-ll/pmc74283ey/auh/3490/c.eli.vun'], ['-lr/pmc75500ey/auh/4240/c.eli.vun'], ['-ml/pmc75907ey/auh/4215/c.eli.vun'], ['-mr/pmc70056ey/auh/3600/c.eli.vun'], ['-pl/pmc7

In [ ]:
##Bigram:It is a common type of collocation.Bigrams are two adjacent words, such as 'CT scan', 'machine learning', or 'social media'.

##Bigram AssocMeasures : A collection of bigram association measures. Each association measure is provided as a function with three arguments:
##bigram_score_fn(n_ii, (n_ix, n_xi), n_xx).

##Collocation:Phrases or expressions containing multiple words, that are highly likely to co-occur.

In [ ]:
from nltk.metrics import BigramAssocMeasures
from nltk.collocations import BigramCollocationFinder

bigrams_patterns = []

for token in tokenized_sent :
  for words in token :
    bigram_collocation = BigramCollocationFinder.from_words(words.split('/'))
    # Top 10 most occurring collocations
    bigrams_patterns.append(bigram_collocation.nbest(BigramAssocMeasures.likelihood_ratio, 10))
    print("Bigrams:", bigram_collocation.nbest(BigramAssocMeasures.likelihood_ratio, 10))

Bigrams: []
Bigrams: [('ey972', '11.a6ddd.han')]
Bigrams: [('-pl', 'pmc71439ey'), ('2223', 'c'), ('auh', '2223'), ('pmc71439ey', 'auh')]
Bigrams: []
Bigrams: [('-pr', 'pmc74892ey'), ('2370', 'c'), ('auh', '2370'), ('pmc74892ey', 'auh')]
Bigrams: []
Bigrams: [('-r', 'pmc72439ey'), ('985', 'c.el'), ('auh', '985'), ('pmc72439ey', 'auh')]
Bigrams: []
Bigrams: [('-11p', 'pmc01336ey'), ('1165', 'c'), ('auh', '1165'), ('pmc01336ey', 'auh')]
Bigrams: []
Bigrams: [('-12p', 'pmc75635ey'), ('1860', 'c'), ('auh', '1860'), ('pmc75635ey', 'auh')]
Bigrams: []
Bigrams: [('-13p', 'pmc70016ey'), ('2050', 'c'), ('auh', '2050'), ('pmc70016ey', 'auh')]
Bigrams: []
Bigrams: [('-21p', 'pmc76368ey'), ('2270', 'c'), ('auh', '2270'), ('pmc76368ey', 'auh')]
Bigrams: []
Bigrams: [('-22p', 'pmc72223ey'), ('2375', 'c'), ('auh', '2375'), ('pmc72223ey', 'auh')]
Bigrams: []
Bigrams: [('-23p', 'pmc74605ey'), ('2390', 'c'), ('auh', '2390'), ('pmc74605ey', 'auh')]
Bigrams: []
Bigrams: [('-31p', 'pmc74776ey'), ('2445', 'c

In [ ]:
len(bigrams_patterns)

266

In [ ]:
from nltk.metrics import BigramAssocMeasures
from nltk.collocations import BigramCollocationFinder

bigrams_patterns = []

for token in tokenized_sent :
  for words in token :
    bigram_collocation = BigramCollocationFinder.from_words(words.split('/'))
    # Top 10 most occurring collocations
    bigrams_patterns.append(bigram_collocation.nbest(BigramAssocMeasures.likelihood_ratio, 10))
    print("Bigrams:", bigram_collocation.nbest(BigramAssocMeasures.likelihood_ratio, 10))

Bigrams: []
Bigrams: [('ey972', '11.a6ddd.han')]
Bigrams: [('-al', 'pmc77612ey'), ('1915', 'c.eli.vun'), ('auh', '1915'), ('pmc77612ey', 'auh')]
Bigrams: [('-ar', 'pmc72671ey'), ('1840', 'c.eli.vun'), ('auh', '1840'), ('pmc72671ey', 'auh')]
Bigrams: [('-bl', 'pmc76614ey'), ('1705', 'c.eli.vun'), ('auh', '1705'), ('pmc76614ey', 'auh')]
Bigrams: [('-br', 'pmc76649ey'), ('1705', 'c.eli.vun'), ('auh', '1705'), ('pmc76649ey', 'auh')]
Bigrams: [('-cl', 'pmc75672ey'), ('2275', 'c.eli.hea.vun'), ('auh', '2275'), ('pmc75672ey', 'auh')]
Bigrams: [('-cr', 'pmc76847ey'), ('1855', 'c.eli.vun'), ('auh', '1855'), ('pmc76847ey', 'auh')]
Bigrams: [('-dl', 'pmc74817ey'), ('2490', 'c.eli.elm.hea.vun'), ('auh', '2490'), ('pmc74817ey', 'auh')]
Bigrams: [('-dr', 'pmc01084ey'), ('2460', 'c.eli.vun'), ('auh', '2460'), ('pmc01084ey', 'auh')]
Bigrams: [('-el', 'pmc76217ey'), ('2675', 'c.eli.vun'), ('auh', '2675'), ('pmc76217ey', 'auh')]
Bigrams: [('-er', 'pmc74925ey'), ('2655', 'c.eli.elm.hea.vun'), ('auh', '26

In [ ]:
## Trigram:It is a common type of collocation.Trigrams are three adjacent words,  such as 'out of business', or 'Proctor and Gamble'
## Trigram AssocMeasures:A collection of trigram association measures. Each association measure is provided as a function with four arguments:
##trigram_score_fn(n_iii,(n_iix, n_ixi, n_xii),(n_ixx, n_xix, n_xxi),n_xxx)

In [ ]:
from nltk.collocations import TrigramCollocationFinder
from nltk.metrics import TrigramAssocMeasures

trigrams_pattern = []

for token in tokenized_sent :
  for words in token :
    trigram_collocation = TrigramCollocationFinder.from_words(words.split('/'))
    trigrams_pattern.append(trigram_collocation.nbest(TrigramAssocMeasures.likelihood_ratio,10))
    print("Trigrams:", trigram_collocation.nbest(TrigramAssocMeasures.likelihood_ratio,10))

Trigrams: []
Trigrams: []
Trigrams: [('-pl', 'pmc71439ey', 'auh'), ('auh', '2223', 'c'), ('pmc71439ey', 'auh', '2223')]
Trigrams: []
Trigrams: [('-pr', 'pmc74892ey', 'auh'), ('auh', '2370', 'c'), ('pmc74892ey', 'auh', '2370')]
Trigrams: []
Trigrams: [('-r', 'pmc72439ey', 'auh'), ('auh', '985', 'c.el'), ('pmc72439ey', 'auh', '985')]
Trigrams: []
Trigrams: [('-11p', 'pmc01336ey', 'auh'), ('auh', '1165', 'c'), ('pmc01336ey', 'auh', '1165')]
Trigrams: []
Trigrams: [('-12p', 'pmc75635ey', 'auh'), ('auh', '1860', 'c'), ('pmc75635ey', 'auh', '1860')]
Trigrams: []
Trigrams: [('-13p', 'pmc70016ey', 'auh'), ('auh', '2050', 'c'), ('pmc70016ey', 'auh', '2050')]
Trigrams: []
Trigrams: [('-21p', 'pmc76368ey', 'auh'), ('auh', '2270', 'c'), ('pmc76368ey', 'auh', '2270')]
Trigrams: []
Trigrams: [('-22p', 'pmc72223ey', 'auh'), ('auh', '2375', 'c'), ('pmc72223ey', 'auh', '2375')]
Trigrams: []
Trigrams: [('-23p', 'pmc74605ey', 'auh'), ('auh', '2390', 'c'), ('pmc74605ey', 'auh', '2390')]
Trigrams: []
Trigr

In [ ]:
len(trigrams_pattern)

266

In [ ]:
test_error = test_df[test_df['Label']=='__label__error']

In [ ]:
test_no_error = test_df[test_df['Label']== '__label__no_error']

In [ ]:
test_no_error

,Message,Label,Label_num
2,SI,__label__no_error,0
3,BULK/AUH/1764/C,__label__no_error,0
4,END,__label__no_error,0
5,CPM,__label__no_error,0
6,EY015/27.A6BMB.AUH,__label__no_error,0
...,...,...,...
136,14/Y2 MIX,__label__no_error,0
137,15R/P2 MIX,__label__no_error,0
138,12L/G1 MIX,__label__no_error,0
139,12R/S MIX,__label__no_error,0


In [ ]:
test_error

,Message,Label,Label_num
0,-41P/PMC74049EY/AUH/3576/C/LD,__label__error,1
1,-42P/PMC74557EY/AUH/3652/C/LD,__label__error,1
122,-44L/AKE25133EY/AUH/1210/C.HE...,__label__error,1
123,HEA.PEP,__label__error,1
140,BULK,__label__error,1
141,BJ-09//G2 -09//G1-06//R2-05,__label__error,1
143,"""""",__label__error,1


In [ ]:
test_df

,Message,Label
0,-41P/PMC74049EY/AUH/3576/C/LD,__label__error
1,-42P/PMC74557EY/AUH/3652/C/LD,__label__error
2,SI,__label__no_error
3,BULK/AUH/1764/C,__label__no_error
4,END,__label__no_error
...,...,...
139,12R/S MIX,__label__no_error
140,BULK,__label__error
141,BJ-09//G2 -09//G1-06//R2-05,__label__error
142,END OF MESSAGE,__label__no_error


In [ ]:
test_set[0] = {'a': '-41P/PMC74049EY/AUH/3576/'}

In [ ]:
test_set

[{'a': '-41P/PMC74049EY/AUH/3576/'},
 {'a': '-42P/PMC74557EY/AUH/3652/C/LD '},
 {'a': 'SI '},
 {'a': 'BULK/AUH/1764/C '},
 {'a': 'END '},
 {'a': 'CPM '},
 {'a': 'EY015/27.A6BMB.AUH '},
 {'a': '-21P/N '},
 {'a': '-22P/PMC71680EY/AUH/1820/X...'},
 {'a': '-23P/PMC76052EY/AUH/1869/M '},
 {'a': '-31P/PMC75338EY/AUH/2340/X...'},
 {'a': '-32P/N '},
 {'a': '-41L/AKE26177EY/AUH/113/C '},
 {'a': '-41R/N '},
 {'a': '-42L/AKE23927EY/AUH/609/M '},
 {'a': '-42R/AKE29769EY/AUH/953/M '},
 {'a': '-43/ALF41072EY/AUH/1534/M '},
 {'a': '-44L/600/E.EIC '},
 {'a': '-44R/AKE23963EY/AUH/778/M '},
 {'a': '-51/N '},
 {'a': '-52/N '},
 {'a': '-53/N '},
 {'a': 'SI  44L  FSK '},
 {'a': 'CPM '},
 {'a': 'EY917/08.A6DDB.AUH '},
 {'a': '-AL/PAJ10367EY/AMS/2207/C....'},
 {'a': '-AR/PMC70521EY/AMS/1925/C '},
 {'a': '-BL/HML00610EJ/AMS/2850/C....'},
 {'a': '-BR/PMC77052EY/AMS/1505/C....'},
 {'a': '-CL/HML00662EJ/AMS/2710/C....'},
 {'a': '-CR/PMC73662EY/AMS/1780/C....'},
 {'a': '-DL/HML00683EJ/AMS/2710/C....'},
 {'a': '-D

In [ ]:
## Splitting in bigram and trigram for the test data.

In [ ]:
sentences = test_no_error['Message'].tolist()

tokenized_sent = []
for s in sentences:
    tokenized_sent.append(word_tokenize(s.lower()))
print(tokenized_sent)

[['si'], ['bulk/auh/1764/c'], ['end'], ['cpm'], ['ey015/27.a6bmb.auh'], ['-21p/n'], ['-22p/pmc71680ey/auh/1820/x', '...'], ['-23p/pmc76052ey/auh/1869/m'], ['-31p/pmc75338ey/auh/2340/x', '...'], ['-32p/n'], ['-41l/ake26177ey/auh/113/c'], ['-41r/n'], ['-42l/ake23927ey/auh/609/m'], ['-42r/ake29769ey/auh/953/m'], ['-43/alf41072ey/auh/1534/m'], ['-44l/600/e.eic'], ['-44r/ake23963ey/auh/778/m'], ['-51/n'], ['-52/n'], ['-53/n'], ['si', '44l', 'fsk'], ['cpm'], ['ey917/08.a6ddb.auh'], ['-al/paj10367ey/ams/2207/c', '....'], ['-ar/pmc70521ey/ams/1925/c'], ['-bl/hml00610ej/ams/2850/c', '....'], ['-br/pmc77052ey/ams/1505/c', '....'], ['-cl/hml00662ej/ams/2710/c', '....'], ['-cr/pmc73662ey/ams/1780/c', '....'], ['-dl/hml00683ej/ams/2710/c', '....'], ['-dr/pmc79883ey/ams/2130/c'], ['-el/hml00914ej/ams/2710/c', '....'], ['-er/pmc72276ey/ams/2340/c'], ['-fl/hml00920ej/ams/2710/c', '....'], ['-11p/pmc71010ey/man/1274/c'], ['-12l/ake/man/551/bj1-12r/n'], ['-13l/ake61670ey/man/692/by', '...'], ['-14l/ake7

In [ ]:
sentences = cpm_incorrect_format_txt[0].tolist()

tokenized_sent = []
for s in sentences:
    tokenized_sent.append(word_tokenize(s.lower()))
print(tokenized_sent)

[['cpm'], ['ey956/28.a6ddd.auh'], ['-al/paj10229ey/jnb/2280/c.elm'], ['-ar/pmc77270ey/jnb/1540/c'], ['-bl/pmc77913ey/jnb/1235/c'], ['-br/pmc77717ey/jnb/1250/c'], ['-cl/pmc70333ey/jnb/1420/c'], ['-cr/pmc75544ey/jnb/1440/c.eli'], ['-dl/pmc76410ey/jnb/1475/c'], ['-dr/pmc76297ey/jnb/1605/c'], ['-el/pmc77378ey/jnb/1770/c'], ['-er/pmc76550ey/nbo/1820/c'], ['-fl/pmc72704ey/jnb/1920/c.rfl.eli'], ['-fr/pmc74296ey/jnb/2085/c'], ['-gl/pmc74420ey/jnb/2330/c'], ['-gr/pmc77514ey/jnb/2245/c'], ['-hl/pmc74075ey/jnb/5105/c'], ['-hr/pmc75724ey/jnb/4850/c.eli'], ['-jl/pmc74745ey/jnb/2525/c.eli'], ['-jr/pmc73455ey/jnb/2375/c'], ['-kl/pmc79972ey/jnb/2745/c.eli.vun'], ['-kr/pmc01321ey/jnb/2760/c'], ['-ll/amj61411fx/jnb/3430/c'], ['-lr/pmc77668ey/jnb/2825/c.eli.vun'], ['-ml/pmc76713ey/jnb/3410/c'], ['-mr/pmc76683ey/jnb/3230/c'], ['-pl/pmc70975ey/jnb/2605/c.eli.elm'], ['-pr/pmc72324ey/jnb/3325/c'], ['-r/pmc71330ey/jnb/2995/c'], ['-11p/pmc76662ey/jnb/1040/c'], ['-12p/pmc74847ey/jnb/1670/c.eli.vun'], ['-13p/pmc

In [ ]:
bigrams_patterns_test = []

for token in tokenized_sent :
  for words in token :
    bigram_collocation = BigramCollocationFinder.from_words(words.split('/'))
    # Top 10 most occurring collocations
    bigrams_patterns_test.append(bigram_collocation.nbest(BigramAssocMeasures.likelihood_ratio, 10))
    print("Bigrams:", bigram_collocation.nbest(BigramAssocMeasures.likelihood_ratio, 10))

Bigrams: []
Bigrams: [('1764', 'c'), ('auh', '1764'), ('bulk', 'auh')]
Bigrams: []
Bigrams: []
Bigrams: [('ey015', '27.a6bmb.auh')]
Bigrams: [('-21p', 'n')]
Bigrams: [('-22p', 'pmc71680ey'), ('1820', 'x'), ('auh', '1820'), ('pmc71680ey', 'auh')]
Bigrams: []
Bigrams: [('-23p', 'pmc76052ey'), ('1869', 'm'), ('auh', '1869'), ('pmc76052ey', 'auh')]
Bigrams: [('-31p', 'pmc75338ey'), ('2340', 'x'), ('auh', '2340'), ('pmc75338ey', 'auh')]
Bigrams: []
Bigrams: [('-32p', 'n')]
Bigrams: [('-41l', 'ake26177ey'), ('113', 'c'), ('ake26177ey', 'auh'), ('auh', '113')]
Bigrams: [('-41r', 'n')]
Bigrams: [('-42l', 'ake23927ey'), ('609', 'm'), ('ake23927ey', 'auh'), ('auh', '609')]
Bigrams: [('-42r', 'ake29769ey'), ('953', 'm'), ('ake29769ey', 'auh'), ('auh', '953')]
Bigrams: [('-43', 'alf41072ey'), ('1534', 'm'), ('alf41072ey', 'auh'), ('auh', '1534')]
Bigrams: [('-44l', '600'), ('600', 'e.eic')]
Bigrams: [('-44r', 'ake23963ey'), ('778', 'm'), ('ake23963ey', 'auh'), ('auh', '778')]
Bigrams: [('-51', 'n'

In [ ]:
bigrams_patterns_incorr = []

for token in tokenized_sent :
  for words in token :
    bigram_collocation = BigramCollocationFinder.from_words(words.split('/'))
    # Top 10 most occurring collocations
    bigrams_patterns_incorr.append(bigram_collocation.nbest(BigramAssocMeasures.likelihood_ratio, 10))
    print("Bigrams:", bigram_collocation.nbest(BigramAssocMeasures.likelihood_ratio, 10))

Bigrams: []
Bigrams: [('ey956', '28.a6ddd.auh')]
Bigrams: [('-al', 'paj10229ey'), ('2280', 'c.elm'), ('jnb', '2280'), ('paj10229ey', 'jnb')]
Bigrams: [('-ar', 'pmc77270ey'), ('1540', 'c'), ('jnb', '1540'), ('pmc77270ey', 'jnb')]
Bigrams: [('-bl', 'pmc77913ey'), ('1235', 'c'), ('jnb', '1235'), ('pmc77913ey', 'jnb')]
Bigrams: [('-br', 'pmc77717ey'), ('1250', 'c'), ('jnb', '1250'), ('pmc77717ey', 'jnb')]
Bigrams: [('-cl', 'pmc70333ey'), ('1420', 'c'), ('jnb', '1420'), ('pmc70333ey', 'jnb')]
Bigrams: [('-cr', 'pmc75544ey'), ('1440', 'c.eli'), ('jnb', '1440'), ('pmc75544ey', 'jnb')]
Bigrams: [('-dl', 'pmc76410ey'), ('1475', 'c'), ('jnb', '1475'), ('pmc76410ey', 'jnb')]
Bigrams: [('-dr', 'pmc76297ey'), ('1605', 'c'), ('jnb', '1605'), ('pmc76297ey', 'jnb')]
Bigrams: [('-el', 'pmc77378ey'), ('1770', 'c'), ('jnb', '1770'), ('pmc77378ey', 'jnb')]
Bigrams: [('-er', 'pmc76550ey'), ('1820', 'c'), ('nbo', '1820'), ('pmc76550ey', 'nbo')]
Bigrams: [('-fl', 'pmc72704ey'), ('1920', 'c.rfl.eli'), ('jnb',

In [ ]:
from nltk.collocations import TrigramCollocationFinder
from nltk.metrics import TrigramAssocMeasures

trigrams_pattern_test = []

for token in tokenized_sent :
  for words in token :
    trigram_collocation = TrigramCollocationFinder.from_words(words.split('/'))
    trigrams_pattern_test.append(trigram_collocation.nbest(TrigramAssocMeasures.likelihood_ratio,10))
    print("Trigrams:", trigram_collocation.nbest(TrigramAssocMeasures.likelihood_ratio,10))

Trigrams: []
Trigrams: []
Trigrams: [('-al', 'paj10229ey', 'jnb'), ('jnb', '2280', 'c.elm'), ('paj10229ey', 'jnb', '2280')]
Trigrams: [('-ar', 'pmc77270ey', 'jnb'), ('jnb', '1540', 'c'), ('pmc77270ey', 'jnb', '1540')]
Trigrams: [('-bl', 'pmc77913ey', 'jnb'), ('jnb', '1235', 'c'), ('pmc77913ey', 'jnb', '1235')]
Trigrams: [('-br', 'pmc77717ey', 'jnb'), ('jnb', '1250', 'c'), ('pmc77717ey', 'jnb', '1250')]
Trigrams: [('-cl', 'pmc70333ey', 'jnb'), ('jnb', '1420', 'c'), ('pmc70333ey', 'jnb', '1420')]
Trigrams: [('-cr', 'pmc75544ey', 'jnb'), ('jnb', '1440', 'c.eli'), ('pmc75544ey', 'jnb', '1440')]
Trigrams: [('-dl', 'pmc76410ey', 'jnb'), ('jnb', '1475', 'c'), ('pmc76410ey', 'jnb', '1475')]
Trigrams: [('-dr', 'pmc76297ey', 'jnb'), ('jnb', '1605', 'c'), ('pmc76297ey', 'jnb', '1605')]
Trigrams: [('-el', 'pmc77378ey', 'jnb'), ('jnb', '1770', 'c'), ('pmc77378ey', 'jnb', '1770')]
Trigrams: [('-er', 'pmc76550ey', 'nbo'), ('nbo', '1820', 'c'), ('pmc76550ey', 'nbo', '1820')]
Trigrams: [('-fl', 'pmc727

In [ ]:
indi_patt = []

for pat in bigrams_patterns :
  for indi_pat in pat :
    for i in indi_pat :
      indi_patt.append(i)

In [ ]:
indi_patt_test = []

for pat in bigrams_patterns_test :
  for indi_pat in pat :
    for i in indi_pat :
      indi_patt_test.append(i)

In [ ]:
indi_patt[0:5]

['ey972', '11.a6ddd.han', '-pl', 'pmc71439ey', '2223']

In [ ]:
indi_patt_test[0:5]

['1764', 'c', 'auh', '1764', 'bulk']

In [ ]:
indi_patt = []

for pat in bigrams_patterns :
  for indi_pat in pat :
    for i in indi_pat :
      indi_patt.append(i)

In [ ]:
indi_patt_incr = []

for pat in bigrams_patterns_incorr :
  for indi_pat in pat :
    for i in indi_pat :
      indi_patt_incr.append(i)

In [ ]:
#!pip install --target=$nb_path fuzzywuzzy

In [ ]:
##FuzzyWuzzy is a library of Python which is used for string matching. Fuzzy string matching is the process of finding strings that match a given pattern. Basically it uses Levenshtein Distance to calculate the differences between sequences.
##Fuzzy string matching is the technique of finding strings that match a pattern approximately (rather than exactly). In another word, fuzzy string matching is a type of search that will find matches even when users misspell words or enter only partial words for the search.

In [ ]:
from fuzzywuzzy import fuzz

/content/notebooks/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [ ]:
##Fuzz Ratio : This ratio uses a simple technique which involves calculating the edit distance (Levenshtein distance) between two strings. The unique feature of 'fuzz. ratio' lies in the fact that it takes into consideration minimal differences existing between both strings.

In [ ]:
## If the Message contains incorrect format , then we get less fuzz score.
fuzz.partial_ratio(indi_patt,indi_patt_incr)

7

In [ ]:
trigrams_pattern_test = []

for token in tokenized_sent :
  for words in token :
    trigram_collocation = TrigramCollocationFinder.from_words(words.split('/'))
    trigrams_pattern_test.append(trigram_collocation.nbest(TrigramAssocMeasures.likelihood_ratio,10))
    print("Trigrams:", trigram_collocation.nbest(TrigramAssocMeasures.likelihood_ratio,10))

In [ ]:
fuzz.ratio(indi_patt,indi_patt_test)

3

In [ ]:
##The partial ratio helps us to perform substring matching. This takes the shortest string and compares it with all the substrings of the same length.

In [ ]:
fuzz.partial_ratio(indi_patt,indi_patt_test)

32

In [ ]:
##In token sort ratio, the strings are tokenized and pre-processed by converting to lower case and getting rid of punctuation. The strings are then sorted alphabetically and joined together. Post this, the Levenshtein distance similarity ratio is calculated between the strings.

In [ ]:
fuzz.token_sort_ratio(indi_patt,indi_patt_test)

14

In [ ]:
##random shuffle : The shuffle() is an inbuilt method of the random module. It is used to shuffle a sequence (list). Shuffling a list of objects means changing the position of the elements of the sequence using Python.
##Syntax: random.shuffle(sequence, function)
##Parameters:
#  sequence : can be a list
#  function : optional and by default is random(). It should return a value between 0 and 1.


In [ ]:
## Random Shuffling
import random

indi_patt = random.shuffle(indi_patt)
indi_patt_test = random.shuffle(indi_patt_test)

In [ ]:
fuzz.ratio(indi_patt,indi_patt_test)

0

In [ ]:
fuzz.partial_ratio(indi_patt,indi_patt_test)

0

In [ ]:
fuzz.token_sort_ratio(indi_patt,indi_patt_test)

0

In [ ]:
#############Trigram Fuzzy Score Methodology###############################

In [ ]:
indi_patt = []

for pat in trigrams_pattern :
  for indi_pat in pat :
    for i in indi_pat :
      indi_patt.append(i)

In [ ]:
indi_patt_test = []

for pat in trigrams_pattern_test :
  for indi_pat in pat :
    for i in indi_pat :
      indi_patt_test.append(i)

In [ ]:
fuzz.ratio(indi_patt,indi_patt_test)

2

In [ ]:
fuzz.partial_ratio(indi_patt,indi_patt_test)

28

In [ ]:
fuzz.token_sort_ratio(indi_patt,indi_patt_test)

19

In [ ]:
## Random Shuffling
import random

indi_patt = random.shuffle(indi_patt)
indi_patt_test = random.shuffle(indi_patt_test)

In [ ]:
fuzz.ratio(indi_patt,indi_patt_test)

0

In [ ]:
fuzz.partial_ratio(indi_patt,indi_patt_test)

0

In [ ]:
fuzz.token_sort_ratio(indi_patt,indi_patt_test)

0

In [ ]:
####################################### NLTK Based Information Extraction ###########################################

In [ ]:
import nltk

In [ ]:
train_set = []

for i in range(0,len(train_df)):
  train_set.append(list((dict(a=train_df['Message'][i]),train_df['Label'][i])))

In [ ]:
test_set = []

for i in range(0,len(test_df)):
  test_set.append((dict(a=test_df['Message'][i])))

In [ ]:
classifier = nltk.classify.NaiveBayesClassifier.train(train_set)

In [ ]:
sorted(classifier.labels())

['__label__error', '__label__no_error']

In [ ]:
classifier.show_most_informative_features()

Most Informative Features


In [ ]:
from nltk.classify import maxent

In [ ]:
encoding = maxent.TypedMaxentFeatureEncoding.train(train_set, count_cutoff=1, alwayson_features=True)

In [ ]:
classifier = maxent.MaxentClassifier.train(train_set, bernoulli=False, encoding=encoding, trace=0)

In [ ]:
classifier.show_most_informative_features()

   9.210 a=='-AL/TAJ10229EY/JNB/2280/C.ELM ' and label is '__label__error'
   9.210 a=='-FR/TMC74296EY/JNB/2085. ' and label is '__label__error'
   9.210 a=='-GL/PMC74420EY/JNB/2330/C ' and label is '__label__error'
   9.210 a=='-GR/TMC77514EY/JNB/2245/C ' and label is '__label__error'
   9.210 a=='-HL/PMC74075EY/JNB/5105/C ' and label is '__label__error'
   9.210 a=='-HR/PMC75724EY/JNB/4850/C.ELI ' and label is '__label__error'
   9.210 a=='-JL/PMC74745EY/JNB/2525/C.ELI ' and label is '__label__error'
   9.210 a=='-JR/PMC73455EY/JNB/2375/C ' and label is '__label__error'
   9.210 a=='-KR/PMC01321EY/JNB/2760/C ' and label is '__label__error'
   9.210 a=='-LL/AMJ61411FX/JNB/3430/C ' and label is '__label__error'


In [ ]:
from sklearn.metrics import f1_score

In [ ]:
y_predict = classifier.classify_many(test_set)

In [ ]:
y_true = test_df['Label']

In [ ]:
f1_score(y_true,y_predict, average='weighted')

0.9276888098062476

In [ ]:
from sklearn.metrics import precision_recall_fscore_support

In [ ]:
precision_recall_fscore_support(y_true,y_predict, average='weighted')

/content/notebooks/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(0.9051408179012346, 0.9513888888888888, 0.9276888098062476, None)

In [ ]:
##########################################Conclusion######################################################################################
##Model 1 built using Fast text library provides a higher accuracy of 95% compared to Model 2 built using Ridge Classification algorithm.
##Model 3 is an alternate approach to solve the problem.This model makes use of Bigram and Trigram concepts.

In [ ]:
#################################################################################### The End ##################################################################